<a href="https://colab.research.google.com/github/Jaeger47/DACER-OBJECT-DETECTION/blob/main/Dacer_Object_Detection_Traning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train a custom object detection model with TensorFlow Lite Model Maker

In this colab notebook, you'll learn how to use the [TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/guide/model_maker) to train a custom object detection model to detect Android figurines and how to put the model on a Raspberry Pi.

The Model Maker library uses *transfer learning* to simplify the process of training a TensorFlow Lite model using a custom dataset. Retraining a TensorFlow Lite model with your own custom dataset reduces the amount of training data required and will shorten the training time.


## Preparation

### Install the required packages
Start by installing the required packages, including the Model Maker package from the [GitHub repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) and the pycocotools library you'll use for evaluation.

In [1]:
#required due to tensorflow changes and avoid DNN errors
!pip install tensorflow==2.8
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     / 406.6 MB 125.6 MB/s
ERROR: Operation cancelled by user
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following packages will be REMOVED:
  libcudnn8-dev
The following held packages will be changed:
  libcudnn8
The following packages will be upgraded:
  libcudnn8
1 upgraded, 0 newly installed, 1 to remove and 43 not upgraded.
Need to get 430 MB of archives.
After this operation, 3,139 MB disk space will be freed.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  libcudnn8 8.1.0.77-1+cuda11.2 [430 MB]
Fetched 430 MB in 7s (63.2 MB/s)
(Reading database ... 155632 files and directories currently installed.)
Removing libcudnn8-dev (8.0.5.39-1+cuda11.1) ...
(Reading data

In [ ]:
!pip install -q tflite-model-maker
!pip install -q tflite-support

     |████████████████████████████████| 621 kB 5.2 MB/s 
     |████████████████████████████████| 1.1 MB 30.2 MB/s 
     |████████████████████████████████| 840 kB 19.7 MB/s 
     |████████████████████████████████| 87 kB 2.9 MB/s 
     |████████████████████████████████| 3.4 MB 34.3 MB/s 
     |████████████████████████████████| 1.2 MB 39.1 MB/s 
     |████████████████████████████████| 596 kB 45.1 MB/s 
     |████████████████████████████████| 6.3 MB 27.4 MB/s 
     |████████████████████████████████| 213 kB 44.1 MB/s 
     |████████████████████████████████| 1.1 MB 37.4 MB/s 
     |████████████████████████████████| 120 kB 54.9 MB/s 
     |████████████████████████████████| 25.3 MB 88 kB/s 
     |████████████████████████████████| 352 kB 49.0 MB/s 
     |████████████████████████████████| 1.1 MB 36.8 MB/s 
     |████████████████████████████████| 47.6 MB 37 kB/s 
     |████████████████████████████████| 99 kB 7.0 MB/s 
     |████████████████████████████████| 208 kB 53.2 MB/s 


Import the required packages.

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

### Prepare the dataset

**Import your labeled dataset**

you can use LabelImg to label your images: https://github.com/tzutalin/labelImg

**Recommend Data set for Train and Validate**


*   60% Train / 40% Validate
*   70% Train / 30% Validate

How to label: https://towardsdatascience.com/how-to-label-images-for-object-detection-step-by-step-7ee317f98583

## Train the object detection model

### Step 1: Load the dataset

* Images in `train_data` is used to train the custom object detection model.
* Images in `val_data` is used to check if the model can generalize well to new images that it hasn't seen before.

In [ ]:
train_data = object_detector.DataLoader.from_pascal_voc(
    'your_dir/train',
    'your_dir/train',
    ['label_name_1', 'label_name_2']
)

val_data = object_detector.DataLoader.from_pascal_voc(
    'your_dir/validate',
    'your_dir/validate',
    ['label_name_1', 'label_name_2']
)

### Step 2: Select a model architecture

EfficientDet-Lite[0-4] are a family of mobile/IoT-friendly object detection models derived from the [EfficientDet](https://arxiv.org/abs/1911.09070) architecture.

Here is the performance of each EfficientDet-Lite models compared to each others.

| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|--------------------|-----------|---------------|----------------------|
| EfficientDet-Lite0 | 4.4       | 146           | 25.69%               |
| EfficientDet-Lite1 | 5.8       | 259           | 30.55%               |
| EfficientDet-Lite2 | 7.2       | 396           | 33.97%               |
| EfficientDet-Lite3 | 11.4      | 716           | 37.70%               |
| EfficientDet-Lite4 | 19.9      | 1886          | 41.96%               |

<i> * Size of the integer quantized models. <br/>
** Latency measured on Raspberry Pi 4 using 4 threads on CPU. <br/>
*** Average Precision is the mAP (mean Average Precision) on the COCO 2017 validation dataset.
</i>

In this notebook, we use EfficientDet-Lite0 to train our model. You can choose other model architectures depending on whether speed or accuracy is more important to you.

In [ ]:
spec = model_spec.get('efficientdet_lite0')

### Step 3: Train the TensorFlow model with the training data.

* Set `epochs = 20`, which means it will go through the training dataset 20 times. You can look at the validation accuracy during training and stop when you see validation loss (`val_loss`) stop decreasing to avoid overfitting.
* Set `batch_size = 4` here so you will see that it takes 15 steps to go through the 62 images in the training dataset.
* Set `train_whole_model=True` to fine-tune the whole model instead of just training the head layer to improve accuracy. The trade-off is that it may take longer to train the model.

In [ ]:
model = object_detector.create(train_data, model_spec=spec, batch_size=4, train_whole_model=True, epochs=20, validation_data=val_data)

Epoch 1/20
15/15 [==============================] - 77s 914ms/step - det_loss: 1.7041 - cls_loss: 1.1081 - box_loss: 0.0119 - reg_l2_loss: 0.0630 - loss: 1.7671 - learning_rate: 0.0065 - gradient_norm: 1.8855 - val_det_loss: 1.5227 - val_cls_loss: 1.0386 - val_box_loss: 0.0097 - val_reg_l2_loss: 0.0630 - val_loss: 1.5857
Epoch 2/20
15/15 [==============================] - 8s 580ms/step - det_loss: 1.3844 - cls_loss: 0.9285 - box_loss: 0.0091 - reg_l2_loss: 0.0630 - loss: 1.4473 - learning_rate: 0.0049 - gradient_norm: 2.1512 - val_det_loss: 1.1801 - val_cls_loss: 0.7961 - val_box_loss: 0.0077 - val_reg_l2_loss: 0.0630 - val_loss: 1.2430
Epoch 3/20
15/15 [==============================] - 8s 580ms/step - det_loss: 1.1144 - cls_loss: 0.7169 - box_loss: 0.0080 - reg_l2_loss: 0.0630 - loss: 1.1773 - learning_rate: 0.0048 - gradient_norm: 2.6388 - val_det_loss: 0.9293 - val_cls_loss: 0.6015 - val_box_loss: 0.0066 - val_reg_l2_loss: 0.0630 - val_loss: 0.9923
Epoch 4/20
15/15 [===============

### Step 4. Evaluate the model with the validation data.

After training the object detection model using the images in the training dataset, use the 10 images in the validation dataset to evaluate how the model performs against new data it has never seen before.

As the default batch size is 64, it will take 1 step to go through the 10 images in the validation dataset.

The evaluation metrics are same as [COCO](https://cocodataset.org/#detection-eval).

In [ ]:
model.evaluate(val_data)

1/1 [==============================] - 9s 9s/step



{'AP': 0.73107415,
 'AP50': 1.0,
 'AP75': 0.9163916,
 'AP_/android': 0.7438944,
 'AP_/pig_android': 0.71825397,
 'APl': 0.73107415,
 'APm': -1.0,
 'APs': -1.0,
 'ARl': 0.7902778,
 'ARm': -1.0,
 'ARmax1': 0.7659722,
 'ARmax10': 0.7902778,
 'ARmax100': 0.7902778,
 'ARs': -1.0}

### Step 5: Export as a TensorFlow Lite model.

Export the trained object detection model to the TensorFlow Lite format by specifying which folder you want to export the quantized model to. The default post-training quantization technique is [full integer quantization](https://www.tensorflow.org/lite/performance/post_training_integer_quant). This allows the TensorFlow Lite model to be smaller, run faster on Raspberry Pi CPU and also compatible with the Google Coral EdgeTPU.

In [ ]:
model.export(export_dir='.', tflite_filename='android.tflite')

### Step 6:  Evaluate the TensorFlow Lite model.

Several factors can affect the model accuracy when exporting to TFLite:
* [Quantization](https://www.tensorflow.org/lite/performance/model_optimization) helps shrinking the model size by 4 times at the expense of some accuracy drop.
* The original TensorFlow model uses per-class [non-max supression (NMS)](https://www.coursera.org/lecture/convolutional-neural-networks/non-max-suppression-dvrjH) for post-processing, while the TFLite model uses global NMS that's much faster but less accurate.
Keras outputs maximum 100 detections while tflite outputs maximum 25 detections.

Therefore you'll have to evaluate the exported TFLite model and compare its accuracy with the original TensorFlow model.

In [ ]:
model.evaluate_tflite('android.tflite', val_data)

10/10 [==============================] - 30s 3s/step



{'AP': 0.7220042,
 'AP50': 1.0,
 'AP75': 1.0,
 'AP_/android': 0.715818,
 'AP_/pig_android': 0.7281903,
 'APl': 0.7220042,
 'APm': -1.0,
 'APs': -1.0,
 'ARl': 0.7652778,
 'ARm': -1.0,
 'ARmax1': 0.7652778,
 'ARmax10': 0.7652778,
 'ARmax100': 0.7652778,
 'ARs': -1.0}

In [ ]:
# Download the TFLite model to your local computer.
from google.colab import files
files.download('android.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>